Set ENV user_type to properly run the model in the notebook

In [ ]:
import os
os.environ["user_type"] = "terminal"

A short description about this model.

In [ ]:
"""
A model to simulate the spread of panic in a crowd.
"""

We import all necessary modules and functions from other files.

In [ ]:
import math
from lib.agent import DONT_MOVE
from lib.space import neighbor_ratio
from lib.display_methods import RED, GREEN
from lib.model import Model, MBR_ACTION, NUM_MBRS, COLOR, GRP_ACTION
from registry.registry import get_model, get_agent
import random as rand

These are the constants and global variables we used in this model.

In [ ]:
MODEL_NAME = "panic"
PANICKED = "panicked"
DEBUG = False  # turns debugging code on or off
DEBUG2 = False  # turns deeper debugging code on or off

DEF_DIM = 10
WIDTH = "width"
HEIGHT = "height"
DEF_NUM_PEOPLE = DEF_DIM*DEF_DIM
DEF_NUM_PANIC = 0
DEF_NUM_CALM = int(.7 * DEF_NUM_PEOPLE)
DEF_NUM_PANIC = int(.3 * DEF_NUM_PEOPLE)

AGENT_PREFIX = "Agent"
THRESHHOLD = .2

CALM = "Calm"
PANIC = "Panic"
first_period = True

The following functions define some actions that our agents can make.

In [ ]:
def agent_action(agent, **kwargs):
    """
    This is what agents do each turn of the model.
    """
    if DEBUG2:
        print("The agent is called", agent)
    global first_period
    if first_period:
        start_panic(agent.exec_key)
    first_period = False
    ratio = neighbor_ratio(agent, lambda agent: agent.group_name() == PANIC)
    if ratio > THRESHHOLD:
        if DEBUG2:
            print("Changing the agent's group to panic!")
        agent.has_acted = True
        if agent.group_name() == CALM:
            get_model(agent.exec_key).add_switch(str(agent), CALM, PANIC)
    return DONT_MOVE


def start_panic(exec_key):
    maxPosn = panic_grps[CALM][WIDTH] * panic_grps[CALM][HEIGHT]
    num_panic = panic_grps[PANIC][PANICKED]
    for i in range(0, num_panic):
        agent_posn = rand.randint(0, maxPosn)
        agent_name = "Calm" + str(agent_posn)
        agent = get_agent(agent_name, exec_key)
        if agent is not None and agent.group_name() == CALM:
            get_model(exec_key).add_switch(agent_name, CALM, PANIC)

This structure defines the groups that characterize our agents.

In [ ]:
panic_grps = {
    CALM: {
        GRP_ACTION: None,
        MBR_ACTION: agent_action,
        NUM_MBRS: DEF_NUM_CALM,
        COLOR: GREEN,
        WIDTH: DEF_DIM,
        HEIGHT: DEF_DIM,
    },
    PANIC: {
        GRP_ACTION: None,
        MBR_ACTION: agent_action,
        NUM_MBRS: 0,
        PANICKED: DEF_NUM_PANIC,
        COLOR: RED
    },
}

We subclass `Model` to create our own variant of it.

In [ ]:
class Panic(Model):
    def handle_props(self, props):
        super().handle_props(props)
        grid_height = self.props.get("grid_height")
        grid_width = self.props.get("grid_width")
        num_agents = (grid_height * grid_width)
        if DEBUG2:
            print("The grid dimencions are", grid_height * grid_width)
            print("The number of agents is", num_agents)
        ratio_panic = self.props.get("pct_panic") / 100
        self.num_panic = math.floor(ratio_panic * num_agents)
        self.grp_struct[CALM][NUM_MBRS] = int(num_agents)
        self.grp_struct[PANIC][PANICKED] = int(ratio_panic * num_agents)
        self.grp_struct[CALM][WIDTH] = grid_width
        self.grp_struct[CALM][HEIGHT] = grid_height

Here's where we create the model class.

In [ ]:
def create_model(serial_obj=None, props=None):
    """
    This is for the sake of the API server:
    """
    if serial_obj is not None:
        return Panic(serial_obj=serial_obj)
    else:
        return Panic(MODEL_NAME, grp_struct=panic_grps,
                     props=props, random_placing=False)

The main function runs the whole model.

In [ ]:
def main():
    model = create_model()
    model.run()
    return 0

Now just call main!

In [ ]:
if __name__ == "__main__":
    main()